In [ ]:
import sys; sys.path.insert(0, '..')
 
from scripts.python.processing import *

In [ ]:
def run_data_setup_for_prediction_models(dataset_name:str):
    
    pass

In [ ]:
# Global variables (?)
RAW_DATA_PATH = f"../../../rawdata/"
PROC_DATA_PATH = f"../../../procdata/"
RESULTS_DATA_PATH = f"../../../results/"

DATASET_NAME = "RADCURE"
